## Imports

In [16]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=FutureWarning)

%matplotlib inline
import pandas as pd
import numpy as np
import sys, os, time, datetime
import matplotlib.pyplot as plt


sys.path.append('libraries')
import signal_processing as spl 
import prediction_lib as pl
pl = reload(pl)
spl = reload(spl)

In [17]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
import matplotlib.colors as colors
import pandas as pd
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.linear_model import LogisticRegressionCV as LogRegCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.decomposition import PCA
from tqdm import tqdm
import sys
from scipy import optimize

from collections import Counter

from scipy.cluster.vq import kmeans2,vq, whiten
from sklearn.cluster import KMeans
from hmmlearn.hmm import MultinomialHMM
from python_speech_features import mfcc

##  Load in Signal DF

In [18]:
name = name
INPUT_FILE = 'data/%s.csv' % name

In [19]:
char_inps_raw = pd.read_csv(INPUT_FILE, index_col=0)
char_inps_raw['char'] = char_inps_raw['char'].fillna(' ').replace('#SPACE', ' ')
char_inps_raw['peak_diff'] =char_inps_raw['peak_diff'].fillna(0)
char_inps_raw = char_inps_raw.fillna(0)
char_inps_raw.tail(10)

,index,peak time,signal,start time,end time,char,is_space,peak_diff,0,1,...,16791,16792,16793,16794,16795,16796,16797,16798,16799,16800
1042,1086,596.346667,12875.447266,596.311667,596.661667,b,0,0.270667,18.0,20.0,...,98.0,98.0,-60.0,-27.0,88.0,85.0,46.0,-21.0,-80.0,0.0
1043,1087,596.617333,22127.296875,596.582333,596.932333,b,0,0.149333,17.0,13.0,...,9.0,7.0,6.0,10.0,11.0,10.0,10.0,11.0,9.0,0.0
1044,1088,596.766667,2168.157959,596.731667,597.081667,i,0,0.354667,-5.0,4.0,...,-14.0,-16.0,-17.0,-13.0,-13.0,-21.0,-21.0,-14.0,-14.0,-19.0
1045,1089,597.121333,1214.426758,597.086333,597.436333,t,0,0.294000,26.0,26.0,...,48.0,243.0,253.0,147.0,56.0,-3.0,99.0,157.0,53.0,0.0
1046,1090,597.415333,25453.357422,597.380333,597.730333,,1,1.591333,10.0,6.0,...,-15.0,-15.0,-17.0,-25.0,-28.0,-21.0,-17.0,-21.0,-24.0,0.0
1047,1091,599.006667,6431.091797,598.971667,599.321667,h,0,0.247333,-3.0,-1.0,...,13.0,3.0,-6.0,17.0,10.0,-15.0,2.0,24.0,23.0,28.0
1048,1092,599.254000,3091.520508,599.219000,599.569000,o,0,0.462000,13.0,11.0,...,50.0,41.0,34.0,32.0,28.0,21.0,16.0,13.0,10.0,8.0
1049,1093,599.716000,913.455261,599.681000,600.031000,l,0,0.270667,2.0,4.0,...,40.0,-36.0,-24.0,100.0,-11.0,-66.0,54.0,46.0,41.0,33.0
1050,1094,599.986667,17315.521484,599.951667,600.301667,e,0,0.200667,-13.0,-19.0,...,-17.0,-140.0,-119.0,0.0,59.0,63.0,87.0,89.0,61.0,-7.0
1051,1095,600.187333,5176.197266,600.152333,600.502333,,0,0.000000,-18.0,-13.0,...,-12.0,-11.0,-7.0,-4.0,1.0,8.0,5.0,-1.0,2.0,6.0


In [20]:
targ_s = ''.join(char_inps_raw['char'].values)
targ_s

'alice was beginning to get very tired of sitting by her sister on the bank and having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought alice without pictures or conversations so she was considering in her own mind a well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies when suddenly a white rabbit with pink eyes ran close by her there was nothing so very emarkable in that nor did alice think it so very much oh dear i shall be late it occurred to her afterwards she ought to have wondered at this but at the time it all seemed quite natural but when the rabbit actually toook a watch out of its packet and looked at it and then hurried on alice started to her feet and in another moment down went alice after it never once considering burning how in the 

In [21]:
PARAM_RESULTS ={}

## Processing 

In [28]:
RATE = 44100
WHITEN = True
TOL = 1e-2


MAX_COLS = -1
MFCC_START = 0
MFCC_END = -20
NUM_CLUSTERS = 50
N_COMPONENTS = 100
DO_ALL = False


REPEATS = 5
SMOOTHS = map(lambda x : np.exp(x), range(-4, 2))

In [29]:
def build_params(smooth):
    _id = datetime.datetime.now()
    PARAM_RESULTS[_id] = {'RATE' : RATE,
        'TOL' : TOL,
        'INPUT_FILE' : INPUT_FILE,
         'MAX_COLS': MAX_COLS,
          'NUM_CLUSTERS' : NUM_CLUSTERS,
          'N_COMPONENTS' : N_COMPONENTS,
          'MFCC_START' : MFCC_START,
          'MFCC_START' : MFCC_START,
          'WHITEN' : WHITEN,
          'DO_ALL' : DO_ALL,
          'SMOOTH' : smooth,
          'PREDICTION' : None,
          'ACCURACY_WO_SPACES' : None,
         'OVERALL_ACCURACY' : None,
         'SCORE' : None,
         'MODEL' : None
                          
         }
    return PARAM_RESULTS[_id]

def loop():
    
    # Copy and take subset
    char_inps = char_inps_raw[char_inps_raw.columns[:MAX_COLS]].copy()
    
    # Extract features
    cepstrum_df = pl.extract_cepstrum(char_inps, RATE, mfcc_start=MFCC_START, mfcc_end=MFCC_END)
    
    # Cluster the sounds
    cepstrum_df = pl.cluster(cepstrum_df, num_clusters = NUM_CLUSTERS, n_components = N_COMPONENTS)
    
    # Repeat the hmm experiement for each parameter
    for _ in range(REPEATS):
        # For each smoothing parameter to be tested
        for smooth in SMOOTHS:
            params = build_params(smooth)
            
            estimate, acc, acc_wospace, score, hmm = pl.run_hmm(cepstrum_df, targ_s, 
                                                    NUM_CLUSTERS, t_smooth = smooth, tol = TOL,
                                                    do_all = DO_ALL)   
            params['OVERALL_ACCURACY'] = acc
            params['ACCURACY_WO_SPACES'] = acc_wospace
            params['PREDICTION'] = estimate
            params['SCORE'] = score
            params['MODEL'] = hmm
loop()

RESULTS_DF = pd.DataFrame(PARAM_RESULTS).T

RESULTS_DF.to_csv('RESULTS_DF.csv')

Transmission smoothing: 0.0183156388887
Accuracy: 0.447718631179 Without spaces: 0.347928994083
guess:

Transmission smoothing: 0.0497870683679
Accuracy: 0.429657794677 Without spaces: 0.315976331361
guess:

Transmission smoothing: 0.135335283237
Accuracy: 0.481939163498 Without spaces: 0.388165680473
guess:

Transmission smoothing: 0.367879441171
Accuracy: 0.445817490494 Without spaces: 0.352662721893
guess:

Transmission smoothing: 1.0
Accuracy: 0.408745247148 Without spaces: 0.304142011834
guess:

Transmission smoothing: 2.71828182846
Accuracy: 0.269961977186 Without spaces: 0.160946745562
guess:

Transmission smoothing: 0.0183156388887
Accuracy: 0.374524714829 Without spaces: 0.256804733728
guess:

Transmission smoothing: 0.0497870683679
Accuracy: 0.479087452471 Without spaces: 0.377514792899
guess:

Transmission smoothing: 0.135335283237
Accuracy: 0.213878326996 Without spaces: 0.0591715976331
guess:

Transmission smoothing: 0.367879441171
Accuracy: 0.327946768061 Without spaces: 

In [30]:
RESULTS_DF[['ACCURACY_WO_SPACES','OVERALL_ACCURACY', 'SMOOTH']].dropna().astype(float).groupby('SMOOTH').max()
RESULTS_DF[['ACCURACY_WO_SPACES','OVERALL_ACCURACY', 'SMOOTH']].dropna().astype(float).groupby('SMOOTH').mean()
RESULTS_DF[['ACCURACY_WO_SPACES','OVERALL_ACCURACY', 'SMOOTH']].dropna().astype(float).groupby('SMOOTH').var()

# RESULTS_DF

,ACCURACY_WO_SPACES,OVERALL_ACCURACY
SMOOTH,,
0.018316,0.347929,0.447719
0.049787,0.397633,0.500951
0.135335,0.388166,0.481939
0.367879,0.352663,0.445817
1.000000,0.340828,0.434411
2.718282,0.261538,0.341255


,ACCURACY_WO_SPACES,OVERALL_ACCURACY
SMOOTH,,
0.018316,0.234162,0.359189
0.049787,0.263984,0.382826
0.135335,0.219329,0.345627
0.367879,0.220197,0.341065
1.000000,0.214675,0.331686
2.718282,0.133807,0.244867


,ACCURACY_WO_SPACES,OVERALL_ACCURACY
SMOOTH,,
0.018316,0.006142,0.004028
0.049787,0.007637,0.005503
0.135335,0.010777,0.007109
0.367879,0.011306,0.007431
1.000000,0.010827,0.007402
2.718282,0.005428,0.003762


# Visualizing process 

In [ ]:
char_inps = char_inps_raw.copy()
char_inps = char_inps[char_inps.columns[:MAX_COLS]]
char_inps.tail(10)

## Visualize Spaces 

In [ ]:
pl.view_char(char_inps, ' ', limit = 8)

In [ ]:
pl.view_char(char_inps, 'e', limit = 8)

## Get the cepstrum features from the raw signal 

In [ ]:
cepstrum_df = pl.extract_cepstrum(char_inps, rate, mfcc_start=0, mfcc_end=19)
cepstrum_df.head()

## Visualize Cepstrum Features 

In [ ]:
pl.view_char(cepstrum_df, ' ', xlim = 200, limit = 12)

In [ ]:
pl.view_char(cepstrum_df, 'e', xlim = 200, limit = 12)

## Cluster the features

In [ ]:
cepstrum_df = pl.cluster(cepstrum_df, num_clusters = num_clusters, n_components = n_components)
cepstrum_df.head()

In [ ]:
pl.cluster_proportions(cepstrum_df, [' '] + map(lambda x : chr(x + ord('a')),  range(26)), num_clusters = num_clusters)

## Build Transmission Matrix with bigrams

In [ ]:
A_df, n_unique, unique_chars, id_to_char, char_to_id  = pl.build_transmission_full()
A_df.head()

##  Check the proportion of spaces across the k means classes

In [ ]:
space_props= pl.get_props(cepstrum_df, num_clusters, ' ')
_  = space_props.plot(kind='bar', title = 'Space Proportions')

## Build emissions matrix

In [ ]:
Eta = pl.build_eta(cepstrum_df, unique_chars, num_clusters)
Eta

## Get Unigram proportions

In [ ]:
char_counts = pl.get_char_counts(targ_s, unique_chars)
char_counts

## Run HMM

In [ ]:
pl = reload(pl)

In [ ]:
smooth = 0.7
estimate, acc, acc_wospace, score = pl.run_hmm(cepstrum_df, targ_s, num_clusters, t_smooth = smooth, do_all = DO_ALL) 

In [ ]:
targ_s

In [ ]:
smooths = map(lambda x : np.exp(x), range(-10, 10))
# smooths = np.arange(0,2,.1)
for smooth in smooths:
    estimate, acc, acc_wospace, score = pl.run_hmm(cepstrum_df, targ_s, num_clusters, t_smooth = smooth, do_all = True)   